In [ ]:
import json
import requests
import pandas as pd
import numpy as np
import mdtraj as md
import nglview as nv
import os
from scipy.spatial.transform import Rotation as R


In [ ]:
# AlphaFind endpoint 
api='https://api.stage.alphafind-ted.dyn.cloud.e-infra.cz/search'

# TED endpoint
tedapi='https://ted.cathdb.info/api/v1' 
tedfiles=tedapi+'/files'

# AlphaFold DB endpoint
af='https://alphafold.ebi.ac.uk'
affiles=af + '/files'

In [ ]:
tmp=os.environ.get('TMPDIR','/tmp')
tmp

In [ ]:
# limit the AlphaFind search
domain_limit=20

In [ ]:
# our sample query
query='AF-A0A7L0KP91-F1-model_v4'

In [ ]:
# retrieve domains from TED database (just the residue intervals, for the time being)
def get_domains(qry):
    up = qry.split('-')[1]
    j = requests.get(f'{tedapi}/uniprot/summary/{up}').json()
    return {
        d['ted_id'].split('_')[-1] : 
        [ c.split('-') for c in d['chopping'].split('_') ] 
        for d in j['data']
    }

In [ ]:
# domains for the query structure
qchops = get_domains(query)

In [ ]:
qdomains = len(qchops)
qdomains

In [ ]:
# query AlphaFind
# The server works asynchronously, run this cell repeatedly until all results are returned, e.g. getting list of {qdomains} numbers close to {domain_limit}, not all or mostly zeros
bag = [ 
    requests.get(f'{api}?query={query}_TED{domain:02d}&limit={domain_limit}&superposition=True').json()['results']
    for domain in range(1,qdomains+1)
]
[ len(b) for b in bag ]

In [ ]:
# Have you really run the previsous cell several times?
# Getting less than {qdomains * domain_limit} is fine if you know what's going on; relax the condition then

assert sum([ len(b) for b in bag ]) ==  qdomains * domain_limit

In [ ]:
# arrange results into a dict with target structures as key, listing matching domain pairs for each
tdom = {}
for i,qd in enumerate(bag):
    for td in qd:
        s = td['object_id'].split('_')
        target = '_'.join(s[:-1])
        domain=s[-1]
        if not target in tdom: tdom[target] = {}
        tdom[target][domain] = { 'query_domain' : f'TED{i+1:02d}', 
                                 'tm_score' : td['tm_score'],
                                 'rmsd' : td['rmsd'],
                                 'translation_vector' : np.array(td['translation_vector']),
                                 'rotation_matrix': np.array(td['rotation_matrix'])
                               }
    
#tdom

In [ ]:
# count total target domains for TM score calculation
tchops = { t : get_domains(t) for t in tdom.keys() }
tdomains = { k : len(v) for k,v in tchops.items() }
#tdomains

In [ ]:
# target side TM score (reflecting common vs. target size)
bag_tm_t = { t : 1./tdomains[t] * sum([
                1./(1.+v['rmsd']**2)
                for d,v in doms.items()
            ])
           for t,doms in tdom.items()
         }
#bag_tm

In [ ]:
# query side TM score (commmon vs. query size)
bag_tm_q = { t : 1./qdomains * sum([
                1./(1.+v['rmsd']**2)
                for d,v in doms.items()
            ])
           for t,doms in tdom.items()
         }

In [ ]:
# sort and pretty print results
result = pd.DataFrame({
    'target' : tdom.keys(),
    'target tm score' : [ bag_tm_t[t] for t in tdom.keys()],
    'query tm score' : [ bag_tm_q[t] for t in tdom.keys()],
    'target domains #' : [ tdomains[t] for t in tdom.keys()],
    'common domains #' : [ len(tdom[t]) for t in tdom.keys()],
    'domain pairs' : [ ','.join([ f'{v["query_domain"]}-{k}' for k,v in tdom[t].items()]) for t in tdom.keys()]
}).set_index('target').sort_values('query tm score',ascending=False)
result.iloc[:20]

In [ ]:
# From the table above, pick up the target structure and the domains you want to align the query to

# this is nice
tmodel = 'AF-A0A850XH80-F1-model_v4'
tdoms = ('TED02','TED06')

# add weights to the domains
dweight = np.array([.7,.3])

{ d:tdom[tmodel][d] for d in tdoms }

In [ ]:
# not so nice, the alignments of TED04 and TED05 are not compatible

"""
tmodel = 'AF-A0A7L0X0H7-F1-model_v4'
tdoms = ['TED04','TED05']
dweight = np.array([.7,.3])

len(tdoms),len(dweight)
"""

In [ ]:
# make sure no. of weights and domains match, normalize eventually
assert(len(tdoms) == len(dweight))
dweight /= np.sum(dweight)
# dweight

In [ ]:
# retrieve whole query structure from AlphaFold DB
qpdb = f'{tmp}/{query}.pdb'
r = requests.get(f'{affiles}/{query}.pdb')
with open(qpdb,'wb') as p:
    p.write(r.content)

In [ ]:
# the chosen target as well
tpdb = f'{tmp}/{tmodel}.pdb'
r = requests.get(f'{affiles}/{tmodel}.pdb')
with open(tpdb,'wb') as p:
    p.write(r.content)

In [ ]:
# alignment rotations+translations as returned by the server
rot = np.array([tdom[tmodel][d]['rotation_matrix'] for d in tdoms])
trans = np.array([tdom[tmodel][d]['translation_vector'] for d in tdoms])
#rot,trans

In [ ]:
# matrix -> quaternion for easier averaging
quat = np.array([ R.from_matrix(r).as_quat() for r in rot ])
#quat

In [ ]:
# weighted average rotations and translations
aquat = np.sum(quat * dweight.reshape((-1,1)),axis=0)
aquat /= np.sqrt(np.sum(aquat*aquat))
arot = R.from_quat(aquat).as_matrix()
atrans = np.sum(trans * dweight.reshape((-1,1)),axis=0)

In [ ]:
# load the full query and target
qt=md.load_pdb(qpdb)
tt=md.load_pdb(tpdb)

In [ ]:
# apply the alignment coordinate transformation
# XXX: mdtraj works with nm, we have A
tt.xyz = (tt.xyz - atrans/10.0) @ arot

In [ ]:
# Enjoy

v = nv.NGLWidget()
qc = v.add_component(qt)
tc = v.add_component(tt)
qc.clear()
qc.add_representation('ribbon',color='#ffe0e0')
c = [r for d in tdoms for r in qchops[tdom[tmodel][d]['query_domain']] ]
qc.add_representation('ribbon',color='#ff0000',selection=', '.join([ '-'.join(c1) for c1 in c ]))
tc.clear()
tc.add_representation('ribbon',color='#e0ffe0')
#tc.add_representation('ribbon',color='#ffff00',selection='-'.join(tchops[target[0]][target[1]]))
c= [r  for d in tdoms for r in tchops[tmodel][d]]
tc.add_representation('ribbon',color='#00ff00',selection=', '.join([ '-'.join(c1) for c1 in c ]))
v